In [16]:
import string
import numpy as np
import re
import glob
import tqdm

In [2]:
class TextCleaner():
    """
    Thai text cleaner
    """
    def __init__(self):        
        #https://omniglot.com/writing/atikamekw.htm
        self.charchters = ['p', 't','k','s','c','tc', 'm','n','r','h', 'w','a','e','i','o']
        self.punctuation = string.punctuation + '،' + '؛' + '؟' + '؛' + '۔' + '»' + '«' + '-'
        self.one_space_regex = r"\s((\s)(\s+)?)?"
        #self.multiple_space_regex = r"\s+"
        self.text = None

        self.dict_punct = dict(zip(list(self.punctuation), np.repeat(' ', len(self.punctuation))))
        
    def remove_punct(self, text):
        table = str.maketrans(self.dict_punct)
        text = text.translate(table)
        return text
    
    def remove_num(self, text):
        num_pattern = r'[\u06F0-\u06F9]'
        text = re.sub(num_pattern, ' ', text)
        return text
    
    def remove_spaces(self, text):
        try:
            #remove all whitespaces since spaces are considered as a sentence seperator in thai
            text = re.sub(self.one_space_regex, ' ', text)
            text = text if text[0] != ' ' else text[1:]
            text = text if text[-1] != ' ' else text[:-1]
            return text
        
        except IndexError as e:
            return ''
    
    def is_not_fa_token(self, token):
        for ch in set(token.lower()):
            #if ord(ch) < int(self.min_ascii, 16) or ord(ch) > int(self.max_ascii, 16):
            if ch not in self.charchters:
                return False
        return True

    def remove_foreign_lang(self, text):
        clean_text = ''
        for token in text.split():
            if self.is_not_fa_token(token):
                clean_text += ' ' + token
        return clean_text[1:]
    
    def clean_text(self, text):
        text = text.lower()
        text = self.remove_punct(text)
        text = self.remove_num(text)
        text = self.remove_foreign_lang(text)
        text = self.remove_spaces(text)
        return text

In [3]:
# # Open a file: file
# file = open('Pikokw.txt',mode='r')
 
# # read all lines at once
# all_of_it = file.read()
 
# # close the file
# file.close()

In [4]:
# all_of_it

In [5]:
# len(all_of_it)

In [6]:
# cleaner = TextCleaner()

In [7]:
# cleaned = cleaner.clean_text(all_of_it)

In [8]:
# len(cleaned)

In [9]:
# cleaned

In [10]:
class AtikamekwStemmer():
    
    def __init__(self): 
        #nouns
        self.possisive_pref = ['kit','nit','ot' , 'ki','ni','o', 'k','n','o']
        
        
        self.possisive_suff = ['inan' , 'ino' ,  'iwaw' , 'iwa' ,'aw','iriw','nan', 'awik' , 'im','']
        self.plural_suff = ['a','ak','ok','']
        self.diminutif_suff = ['icic', 'ocic','']
        self.locative_suff = ['ik', 'ok','']
        self.obviativ_suff = ['iw','riw','a','']
        self.surobviatif_suff = ['iriw','riw','']
        self.derivation_suff = ['an','ekin','apo','api','apiskw','']
        
        self.noun_Suffix = []
        
        
        
        # pronouns
        
        self.perosonal_pron = ['nin' , 'kir' , 'wir']
        self.indefinant_pron = ['awik' , 'kekwan' , 'kotak']
        self.interogatice_pron = ['awin' , 'kekwan' , 'tan' , 'tanta', 'tante','tan apitc']
        self.personal_pron_other = ['ni','ki','nit','kit']
        self.personal_pron_independant = ['nin','kir','wir','ninan','kirano','kirawaw','wirawaw']
        self.personal_pron_independant_priority = ['ninctam','kirctam','wirctam','nirctaminan','kirctamiwaw','wirctamiwaw']
        
        self.pronouns = []

        # other  particules determinals
        self.demostrative_words = ['nahwe', 'ohwa','ohwe','oma','nahwe','anahwe'
                             ,'naha','anaha','na','ohki','niki','neki'
                             ,'nihe','ohwi','nehe','anehe','nihi','neta','nete']
        
        self.part_time = ['aptic','minawatc','mocak','nama wiskat','nac','kinowec','ko']
        self.part_rept = ['kiapatc','kiapatci','kiapatc peikwa','koski','oscamec','awocamec']
        self.part_comp = ['kekat','wiec','kirowe','nota','orina','tapicoktc','patok','mia','towi']
        self.part_connect = ['acitc','kirica','aima','aric','kaie']
        self.part_sub = ['e','ka','epwamoci','esko','ickwa','kitci','nota','wetci']
        self.part_quan = ['notc','memantcic','micta','tipi','orina','tepirak']
        self.part_act = ['tekaci','pekatc','mamar']
        self.part_prox = ['warowik','pecotcik','ota','ote','neta','nete','nte']
        self.part_oriant = ['icpimik','notc','opimera','okitc','cipa','pitc','nikanik','otananik','orowitimik','pitakamik']
        self.part_inter = ['la','a','aia','taina']
        self.part_interj = ['ekocka','icine']
        self.particules = []
#         self.particules = ['aptic','minawatc','mocak','nama wiskat','nac','kinowec','ko' 
#              ,'kiapatc','kiapatci','kiapatc peikwa','koski','oscamec','awocamec',
#              'kekat','wiec','kirowe','nota','orina','tapicoktc','patok','mia','towi',
#              'acitc','kirica','aima','aric','kaie',
#              'e','ka','epwamoci','esko','ickwa','kitci','nota','wetci',
#              'notc','memantcic','micta','tipi','orina','tepirak',
#              'tekaci','pekatc','mamar'
#              ,'warowik','pecotcik','ota','ote','neta','nete','nte',
#              'icpimik','notc','opimera','okitc','cipa','pitc','nikanik','otananik','orowitimik','pitakamik',
#              'la','a','aia',
#              'ekocka','icine']
        self.male_female_ind = ['iskwew' # female
                                 ,'iriniw' #male
                                 , 'irinikwew' #male or female 
                               ]
        
        self.stop_words = []
        
        
        #verbs
        
        self.vai_01 = ['n','nawaw','nano','nan','wok','riw','riwa']
        self.vai_03 = ['tai','tawaw','tanano','tai','tan','pan','panak','ripan']
        self.vai_09 = ['natake' , 'nawatake','nanotake','natake','nanatake','take','takenak','ritake','ritakena']
        self.vai_10 = ['nakopan','nawakopan','nanokopan','nanakopan','kopan','wakopan','rikopan','rikopana']
        self.vai_11 = ['in','iekw','ikw','ian','ika','tc','tcik','ritci','ritci']
        self.vai_12a = ['ine','iekwe','ikwe','iane','iake','te','wate','rite','ritena']
        #vai_12b = vai_12a
        self.vai_13 =['ipan','iekopan','ikopan','iapan','iakipan','span','waspan','rispan','rispana']
        self.vai_14 = ['wonen','wewokwen','wokwen','wanen','waken','kwen','wekwen','rikwen','rikwena']
        self.vai_15 = ['wopanen','wekopanen','wokopanen','wapanen','wakipanen','kopanen','wakopanen','rikopanen','rikopanena']
        self.vai_17a = ['kw','tan']
        self.vai_17b = ['kan','kekw','tan']
        
        self.vii_01 = ['o','iw','iwa','a','riw','riwa']
        self.vii_03 = ['pan','pana','ripan','ripana']
        self.vii_09 = ['take','takena','ritake','ritakena']
        self.vii_10 = ['kopan','kopana','rikopan','rikopana']
        self.vii_11 = ['k','ki','rik','riki']
        self.vii_12a = ['ke','kawe','rike','rikawe']
        #vi_12b = vi_12a
        self.vii_13 = ['kipan','kipana','rikipan','rikipana']
        self.vii_14 = ['kwen','kwena','rikwen','rikwena','ikwen','ikwena']
        self.vii_15 = ['kopanen','kopanena','rikopanen']

        self.vta_01 = ['in','itin','inan','itinan','inawaw','itinawaw',
                 'imawaw','imikowaw','anano','ikonano','ananowok', 'ikonanowok', 'imananowa', 'imikonano'
                 , 'aw','ikw','awok','ikok','imawa', 'imikw'
                 , 'anan' , 'ikonan' ,'ananak', 'ikonanak','imanana','imikonan'
                 , 'awaw' , 'ikowaw','awawok','ikowawok'
                 ,'ew','ewok','imew','imewok','imeriw','iko','ikowok','eriw','eriwa','ikoriw','ikoriwa']
        self.vta_03 = ['itai', 'ititai','itan', 'itinan','itawaw', 'itinaw','atanano','ikotanano','atananowok','ikotananowok'
                 'atai', 'ikotai','ataiik','ikoktaiik','atan','ikotan','atananak','ikotananak'
                 'atawaw','ikotawaw','atawawok','ikotawawok','epan','epanak','ikopan','ikowapanak'
                 ,'eripan','eripana','ikoripan','ikor']
        self.vta_09 = ['inatake' , 'inanatake', 'inawatake', 'itinatake','itinanatake'
                 ,'itinawatake','ananotake','ananotakenak','ikonanotake','ikonanotakenak'
                 ,'atake', 'atakenak' , 'ikonatakenak', 'ananatake','ananatakenak'
                 ,'ikonanatakenak','awatake','awatakenak','ikotake','ikowatake','ikotakenak',
                 'ikowatakenak','etake','etakena','ikotakena','eritake','eritakena','ikoritake','ikoritakena']
        self.vta_10 = ['inakopan', 'inanakopan','inawakopan','itinakopan','itinanakopan'
                 ,'itinawakopan','ananokopan','ananokopanak','ikonanokopan','ikonanokopanak'
                 ,'akopan','akopanak','ikokopan','ikokopanak','akopan','ananakopank','ikonanakopan'
                 ,'ananakopanak','ikonanakopanak','awakopan','awakopanak','ikokopan'
                 ,'ikokopanak','ikowakopan','ekopan','ekopana','ikokopan','ikokopan','ikowakopan',
                 'ikowakopana','erikopan','erikopana','ikorikopan','ikorikopana']
        self.vta_11 = ['iin','iak','iekw','itan','itak','itakok','imeko',
                 'okw','okok','imoko','imitako','itokw','itokok','imitoko'
                 ,'ak','akik','imaki','itc','itcik','imitci','akitc','imitc'
                 ,'akitcik','imitcik','imakitci','imtci','atc','iskw',
                 'atcik','imatci','iskik','imiski','itakw','itakok','ekw','ekok',
                 'atc','atcik','imatc','imatcik','imaritci','eritci','ewaritci',
                 'ikoritci','ikowaritci','kotc','ikotcik']
        self.vta_12 = ['iine','itaane','iake','iekwe','itake','itakokwe',
                 'okwe','okwawe','itokwe','itokwawe','amake','akawe',
                 'ikote','ikowate','imite','imitawe','ate','akite','atawe','iske','iskawe'
                 ,'ewekwe','etawe','itakwe','itakawe','amate','awate','ikote','ikowate',
                 'arite','ariwate','ariwatena','ikorite','ikoriwate']
        self.vta_13 = ['ipan','iekopan','iakipan','iekopan','itapan','itakipan','itakokipan'
                 , 'itakipan','okopan','okwapan','itakopan','itokwapan','akipan','akwapan','ispan',
                 'iwaspan','akitipan','akitwapan','atipan','atwapan','ewekopan','ewekwapan','aspan',
                 'aspana','awaspan','awaspana','arispan','arispana','imitipan','imitwapan','iskipan',
                 'iskwapan','itakopan','itakwapan','ikospan','ikowaspana',
                 'ikorispan','ikorwiwarispan']
        self.vta_14 = ['iwonen','iwakwen','iwekwen','itawonen','itakokwen','owokwen', 'owokwenak'
                 ,'awoken','awokenak','awakiten','awakitenak','awoten','awotenak','ewekwen','ewekwenak',
                 'akwen','awakwen','itokwen','itokwenak','ikwen','iwakwen','itkowen','itokwenak',
                 'iskwen','iskwaken','itakwen','itawenak','ikokwen','ikowakwen',
                 'ikowakwen','arikwen','arikwena','ikorikwen','ikorikwena']
        self.vta_15 = ['iwopanen','iakopanen','iekopanen','itawopanen','itakopanen',
                 'owokopanen','awokopanenak','awokipanen','awokipanenak','awokitipanen',
                 'awokitipanenak','awotipanen','awotipanenak','ewekopanen','ewekopanenak',
                 'ikokopanenak','ikokopanen','iskwakopanen','iskopanen','iamitokopanenak','iamitokopanen',
                 'iwakopanen','ikopanen','itokokopanenak','itokokopanen']
        self.vta_17_a = ['ici','icinan','icikw','imik','imakw','atan','atanak','imatan','i','ik','im','akw','akok']
        self.vta_17_b  = ['icikan','icikanan','icikekw','imakanak','imakan','akan','akanak','imakan','akekw','akekok']
        
        self.vti_01 = ['en','enawaw','enano','enan','am','amok','amiriw','amiriwa','w','wa','riw','riwawa']
        self.vti_03 = ['etai','etawaw','etano','etai','etan','amopan','amopank','amiripan','amiripana','pan','pana','ripan','ripana']
        self.vti_09 = ['enatake','enawatake','enanotake','enanatake','amotake','amotakenak','amiritake',
                 'amiritakena','take','takena','ritakena','ritakena']
        self.vti_10 = ['enakopan','enawakopan','enanokopan','enanakopan','amokopan','amowakopan','amirikopan','amirikopana']
        self.vti_11 = ['aman','amekw','amokw','amak','ak','akik','amiritci','k','ki','riki']
        self.vti_12a = ['amane','amekwe','amiekwe','amokwe','amoikwe','amane','amake','amane','amote','amowate','amirite','amiritena']
        #vti_12b = vit_12a
        self.vti_13 = ['amopan','amekopan','amokopan','amapan','amakipan','akipan','akwapan',
                  'amirispan', 'amirispana']
        self.vti_14 = ['amowonen','amwewokwen','amowokwen','amowaken','amokwen','amowakwen','amirikwen','amirikwena']
        self.vti_15 = ['amowopanen','amokwekopanen','amokopanen','amowapanen','amowakipanen','amokopanen','amowkopanen','amirikopanen','amirikopanena']
        self.vti_17a = ['a','amokw','eta']
        self.vti_17b = ['ekan','amokan']
#         self.vti2_01 = ['n','nawaw','nano','nan','w','wok','riw','riwa','o','iw','iwa']
#         self.vti2_03 = ['tai','tawaw','tanano','tai','tan','pan','panak','ripan','ripana','pan','ipana','opan','opana']
#         self.vti2_09 = ['natake','nawatake','nanotake',
#                   'nanatake','take','takenak','ritake',
#                  'ritakena','otake','otakena','oritakena','oritakena']
#         self.vti2_10 = ['nakopan','nawakopan','nanokopan','nakopan','kopan','wakopan','rikopan','rikopana']
#         self.vti2_11 = ['in','iekw','ikw','ian','iak','tc','tcik','ritci','ki','niki','nik']
#         self.vti2_12a = ['ine','iekwe','ikwe','iane','iake','te','wate','rite','ritena']
#         #vti_12b = vit_12a
#         self.vti_13 = ['ipan','iekopan','ikopan','iapan','iakipan','span','waspan',
#                   'rispan', 'rispana']
#         self.vti_14 = ['wonen','wewokwen','wokwen','wanen','waken','kwen','wakwen','rikwena','rikwena']
#         self.vti_15 = ['wopanen','wekopanen','wokopanen','wapanen','wakipanen','kopanen','wakopanen','rikopanen','rikopanena']
#         self.vti_17a = ['kw','tan']
#         self.vti_17b = ['kan','kekw','nano']
        
        self.vai = []
        self.vii = []
        self.vta = []
        self.vti = []
        self.verbs_Suffix = []
        
        self.all_Suffix = []
        
        self.prepare_rules()
        
    def prepare_rules(self):
        # nouns
        isem = []
        for i1 in self.possisive_suff:
            for i2 in self.plural_suff:
                for i3 in self.diminutif_suff:
                    for i4 in self.locative_suff:
                        for i5 in self.obviativ_suff:
                            for i6 in self.surobviatif_suff:
                                for i7 in self.derivation_suff:
                                    i = i1 + i2 + i3 + i4 + i5 + i6 + i7
                                    isem.append(i)
        #self.noun_Suffix = list(sorted (list(set(isem)),key = len , reverse = True))
        self.noun_Suffix = isem
        #print(self.noun_Suffix)
        
        #print (len(self.noun_Suffix))
        
        
        # pronouns 
        
        self.pronouns = self.perosonal_pron + self.indefinant_pron + self.interogatice_pron + self.personal_pron_other + self.personal_pron_independant + self.personal_pron_independant_priority
        #print (len(self.pronouns))
        # determiners
        
        self.particules = self.part_time + self.part_rept + self.part_comp + self.part_connect+ self.part_sub+self.part_quan +self.part_act +self.part_prox +self.part_oriant +self.part_inter +self.part_interj
        self.stop_words = self.demostrative_words + self.particules + self.male_female_ind
        #print (len(self.determiners))
        
        #verbs
        
        self.vai = self.vai_01 + self.vai_03 + self.vai_09 + self.vai_10 + self.vai_11 + self.vai_12a + self.vai_13 + self.vai_14 + self.vai_15 + self.vai_17a + self.vai_17b
        self.vii = self.vii_01 + self.vii_03 + self.vii_09 + self.vii_10 + self.vii_11 + self.vii_12a + self.vii_13 + self.vii_14 + self.vii_15
        self.vta = self.vta_01 + self.vta_03 + self.vta_09 + self.vta_10 + self.vta_11 + self.vta_12 + self.vta_13 + self.vta_14  + self.vta_15 + self.vta_17_a + self.vta_17_b 
        self.vti = self.vti_01 +self.vti_03 +self.vti_09 +self.vti_10 +self.vti_11+self.vti_12a +self.vti_13 +self.vti_14 +self.vti_15 +self.vti_17a +self.vti_17b #+self.vti2_01 +self.vti2_03 +self.vti2_09 +self.vti2_10 +self.vti2_11 +self.vti2_12a +self.vti_13 +self.vti_14+self.vti_15 +self.vti_17a +self.vti_17b 
        #self.verbs_Suffix = list(sorted (list(set(self.vai + self.vii + self.vta + self.vti)),key = len , reverse = True))
        self.verbs_Suffix = self.vai + self.vii + self.vta + self.vti
        #print (len(self.verbs_Suffix))
        
        self.all_suffix = list(sorted(list(set(self.noun_Suffix + self.verbs_Suffix)),key = len, reverse = True))
        
        
    def stem_token (self,word):
        
        if word in self.pronouns:
            return "Pron"
        if word in self.stop_words:
            if word in self.demostrative_words:
                return "w_demo"
            if word in self.male_female_ind:
                return "w_m/f"
            if word in self.particules:
                if word in self.part_time:
                    return "part_time"
                if word in self.part_rept:
                    return "part_rept"
                if word in self.part_comp:
                    return "part_comp"
                if word in self.part_connect:
                    return "part_conn"
                if word in self.part_sub:
                    return "part_sub"
                if word in self.part_quan:
                    return "part_quan"
                if word in self.part_act:
                    return "part_act"
                if word in self.part_prox:
                    return "part_prox"
                if word in self.part_oriant:
                    return "part_oriant"
                if word in self.part_inter:
                    return "part_inter"
                if word in self.part_interj:
                    return "part_interj"
        
            return "Det"
        #check prefixes
        for pre in self.possisive_pref:
            n = len(pre)
            try:
                if pre == word[:n]:
                    new_Word = word [n:]
                    if len(new_word)>2:
                        for suff in self.noun_Suffix:
                            m = len(suff)
                            try:
                                if suff == new_Word[len(new_Word)-m:]:
                                    final_word = new_Word[:-m]
                                    if len(final_word)>2:
                                        return final_word
                            except:
                                continue
                    return new_word
            except:
                continue
        
        #check suffixes
        for suff in self.all_suffix:
            n = len(suff)
            try:
                if suff == word[len(word)-n:]:
                    #print(suff)
                    new_word = word[:-n]
                    if len(new_word)>2:
                        if suff in self.vai and new_word[-1] in ['a','e','i']:
                            #print ("vai word")
                            #print(suff)
                            return new_word + 'w'
                        elif  suff in self.vai and new_word[-1] not in ['a','e','i']:
                            #print ("vai word")
                            #print(suff)
                            return new_word 
                        if suff in self.vii:
                            new_word = new_word + 'w'
                            #print ("vii word")
                            #print (suff)
                            return new_word
                        if suff in self.noun_Suffix:
                            #print ("noun word")
                            return new_word   
                        if suff in self.vti:
                            new_word = new_word + 'am'
                            #print ("vti word")
                            #print (suff)
                            return new_word
                        if suff in self.vta and new_word[-1] in ['m','h']:
                            new_word = new_word + 'ew'
                            #print ("vta word")
                            #print (suff)
                            return new_word
                        elif suff in self.vta and new_word[-1] not in ['m','h']:
                            continue          
            except:
                continue
        return word
    
    
    def stem_text(self,text):
        output_text = ""
        for word in text.split(' '):
            new_token = self.stem_token(word)
            output_text = output_text + new_token + ' '
        return output_text[:-1]
       

In [12]:
# AS = AtikamekwStemmer()
# #AS.prepare_rules()
# stemmer = AtikamekwStemmer()
# stemmed = stemmer.stem_text(cleaned)
# print ("befor stemming")
# print (cleaned)
# print("*************************")
# print ("after stemming")
# print(stemmed)
# print(len(stemmed))

## pipeline

In [17]:
def preprocessing_from_folder(input_folder , output_folder):
    files = sorted(glob.glob(input_folder + '/*'))
    cleaned_corpus = ""
    for file in tqdm.tqdm(files):
        print (file)
        file_name  = file.split('\\')[-1]
        #file_comp = file_name_full.split(' ')
        #file_number, file_name = file_comp[0], file_comp[1]
#         print (file_number)
#         print (file_name)
        output_file = output_folder + '/PREPROCESSED_' + file_name
        preprocessing_pipline(file, output_file)

In [20]:
def preprocessing_pipline (file , output_file):
    # Open a file: file
    file = open(file ,mode='r',encoding = 'utf-8')
    all_of_it = file.read()
    file.close()
    
    # our built_in cleaner 
    cleaner = TextCleaner()
    cleaned = cleaner.clean_text(all_of_it)
    
    #word tokenizer
    stemmer = AtikamekwStemmer()
    stemmed = stemmer.stem_text(cleaned)
    #cleaned_text = ""
    #words = tokenize(cleaned)
    #words = word_tokenize(cleaned , keep_whitespace=False,engine = 'attacut')
    #words = deepcut.tokenize(cleaned)
    #for word in words:
        #if (len(word) == 1 or word == " "):
         #   continue
        #else:
         #   cleaned_text+= word + " "
    
    #stopwords removal
    #cleaned_text_nosw = remove_stopwords(cleaned_text)
    
    with open(output_file, 'w',encoding='utf-8') as f:
        f.write(f"{stemmed[:-1]}\n")
    

In [21]:
preprocessing_from_folder('Wikipedia Articales' , 'Wikipedia Articales Processed')

  0%|          | 0/1584 [00:00<?, ?it/s]

Wikipedia Articales\%C3%89cole_Otapi_de_Manawan.txt


  0%|          | 1/1584 [00:00<12:04,  2.19it/s]

Wikipedia Articales\%C3%89cole_secondaire_Otapi.txt


  0%|          | 3/1584 [00:01<07:53,  3.34it/s]

...

100%|█████████▉| 1582/1584 [07:32<00:00,  7.07it/s]

Wikipedia Articales\Pipon.txt


100%|█████████▉| 1583/1584 [07:32<00:00,  4.83it/s]

Wikipedia Articales\atj.txt


100%|██████████| 1584/1584 [10:13<00:00,  2.58it/s]


In [22]:
def make_corpus(input_path, output_file_path):
    file_list = sorted(glob.glob(input_path + '/*'))
    with open(output_file_path, 'a', encoding = 'utf-8') as output_file:
        for file in tqdm.tqdm(file_list):
            with open(file, 'r', encoding = 'utf-8') as input_file:
                file_name_full  = file.split('\\')[-1]
                file_name_without_proces = file_name_full.split('_')[1]
                file_name_without_txt = file_name_without_proces.split('.')[0]
                file_content = input_file.read()
                file_content = file_content.replace('\n', ' ')
                output_file.write(file_name_without_txt)
                output_file.write("$$$")
                output_file.write(file_content)
                output_file.write('\n')

In [23]:
make_corpus ("Wikipedia Articales Processed" , "Atikamekw_corpus.txt")

100%|██████████| 1584/1584 [00:07<00:00, 208.95it/s]


### rules

In [ ]:
# nouns to indicate if the profession is male or female
male_female_words_of_profession = ['iskwew' # female
 ,'iriniw' #male
, 'irinikwew' #male or female 
]

#verbs to indicate  profession
male_female_verbs_of_profession = [ 'iriniwin','iskwewin','riniskwewin']

# indicate the genere of an animal 

male_Female_particular_animals = ['noce' #female
                                 ,'nape' # male
                                 ,]

#NOUNS

# they come by this order po -> p-> di -> loc


possisive_pref = ['kit','nit','ot' , 'ki','ni','o', 'k','n','o']

possisive_suff = ['inan' , 'ino' ,  'iwaw' , 'iwa' ,'aw','iriw','nan', 'awik' , 'im']



plural_suff = ['a','ak','ok>w']

diminutif_suff = ['icic', 'ocic']
#diminutif_suff = ['ic', 'cic']

locative_suff = ['ik', 'ok>w']

#those last two we can add together
obviativ_suff = ['iw','riw','a']

surobviatif = ['iriw','riw']


derivation_suffix = ['an','ekin','apo','api','apiskw']

#PRONOUNS
 
perosonal_pron = ['nin' , 'kir' , 'wir']
indefinant_pron = ['awik' , 'kekwan' , 'kotak']
interogatice_pron = ['awin' , 'kekwan' , 'tan' , 'tanta', 'tante','tan apitc']

personal_pron_prefixes = ['ni','ki','nit','kit']
personal_pron_independant = ['nin','kir','wir','ninan','kirano','kirawaw','wirawaw']
personal_pron_independant_priority = ['ninctam','kirctam','wirctam','nirctaminan','kirctamiwaw','wirctamiwaw']

#
demostrative_words = ['nahwe', 'ohwa','ohwe','oma','nahwe','anahwe'
                     ,'naha','anaha','na','ohki','niki','neki'
                     ,'nihe','ohwi','nehe','anehe','nihi','neta','nete']

#time
#repetation
#comparing
#connectors
#subbordinate + verb
#expressing countite
#Modality of action (way of doing)
#proximity
#spacial-oriantation
#mark of interrogation
#intergection
particules = ['Aptic','minawatc','mocak','nama wiskat','nac','kinowec','ko' 
             ,'kiapatc','kiapatci','kiapatc peikwa','koski','oscamec','awocamec',
             'kekat','wiec','kirowe','nota','orina','tapicoktc','patok','mia','towi',
             'acitc','kirica','aima','aric','kaie',
             'e','ka','epwamoci','esko','ickwa','kitci','nota','wetci',
             'notc','memantcic','micta','tipi','orina','tepirak',
             'tekaci','pekatc','mamar'
             ,'warowik','pecotcik','ota','ote','neta','nete','nte',
             'icpimik','notc','opimera','okitc','cipa','pitc','nikanik','otananik','orowitimik','pitakamik',
             'la','a','aia',
             'ekocka','icine']

# 'kecpin' == if 
# 'kitci' == for
# 'e' == that
# 'wetci' == for
#' tan' == what
#'tan apitc' == when
#'kekwan wetci' = why
# 'aka' == negatuve
# 'ka' == relative

preverbes_for_immidiatly = ['ta','wi','ka','ki']
#VERBS

#verbs_suff = ['an','in', 'w','n','ano','awaw','ok']
# verbs_suff_AI_independent = ['n','nan', 'nano','nawaw','wok'] ->['w']
# verbs_suff_TI_independent = ['en','am','an','ano','awaw','ok'] -> ['am']
# verb_suff_AI_conjunctive = ['n','ian','an','in','iin','tc','ak','iak','iakw','kw','ikw','iekw','ekw','tcik'] ->['w']
# verb_suff_AI_imperative = ['kw','an','kan','kekw','tan'] ->['w']
# verb_suf_TI_imperative = ['amokw','eta','ekan','amokan','ameta'] -> ['am']
# verb_suf_TA_imperative = ['ici','inan','icikw','icinan','icikan','icikanan','icikekw','icikanan'] -> ['ew']



lexical_final = ['ote','ska','kowi','h','aso','acte','ckow','pw']
abstract_final = ['aia','cin','t','n','r','h']

doutfull_suffix = ['take']
subjonctive_suffix = ['e']

imperfect_suffix = ['tai','pan']
past_suffix = 'ki'
future_after_verb = ['ka','kata','ickwa']

preform_lexical = ['miro','kice','matci','micta','mata','kakike']

# reflexiv oneself ['itiso']
# recepcioncal each other ['ito']

#derivation 









VII: verb inanime intransitive

VAI  : anime intransitive

VTA : verb transitive anime

VTI : verb transitive inanime

VTI2 : verb transitive inanime 

#########


01 independant indicatif present

03 independant indicatif impeerfect

09 independant dubitatif present

10 independant dubitatif past

11a conjonctive indicatif present

12a conjonctive subjonctif

12b conjonctive iteratif 

13 conjonctive imperfrect

14 concojonctive dubitatif

15 conjonctive dubitatifg imperfect

17a imperfect indicative present

17b imperfect indicative future

In [2]:
vai_01 = ['n','nawaw','nano','nan','wok','riw','riwa']
vai_03 = ['tai','tawaw','tanano','tai','tan','pan','panak','ripan']
vai_09 = ['natake' , 'nawatake','nanotake','natake','nanatake','take','takenak','ritake','ritakena']
vai_10 = ['nakopan','nawakopan','nanokopan','nanakopan','kopan','wakopan','rikopan','rikopana']
vai_11 = ['in','iekw','ikw','ian','ika','tc','tcik','ritci','ritci']
vai_12a = ['ine','iekwe','ikwe','iane','iake','te','wate','rite','ritena']
#vai_12b = vai_12a
vai_13 =['ipan','iekopan','ikopan','iapan','iakipan','span','waspan','rispan','rispana']
vai_14 = ['wonen','wewokwen','wokwen','wanen','waken','kwen','wekwen','rikwen','rikwena']
vai_15 = ['wopanen','wekopanen','wokopanen','wapanen','wakipanen','kopanen','wakopanen','rikopanen','rikopanena']
vai_17a = ['kw','tan']
vai_17b = ['kan','kekw','tan']

vii_01 = ['o','iw','iwa','a','riw','riwa']
vii_03 = ['pan','pana','ripan','ripana']
vii_09 = ['take','takena','ritake','ritakena']
vii_10 = ['kopan','kopana','rikopan','rikopana']
vi_11 = ['k','ki','rik','riki']
vi_12a = ['ke','kawe','rike','rikawe']
#vi_12b = vi_12a
vi_13 = ['kipan','kipana','rikipan','rikipana']
vi_14 = ['kwen','kwena','rikwen','rikwena','ikwen','ikwena']
vi_15 = ['kopanen','kopanena','rikopanen']


vta_01 = ['in','itin','inan','itinan','inawaw','itinawaw',
         'imawaw','imikowaw','anano','ikonano','ananowok', 'ikonanowok', 'imananowa', 'imikonano'
         , 'aw','ikw','awok','ikok','imawa', 'imikw'
         , 'anan' , 'ikonan' ,'ananak', 'ikonanak','imanana','imikonan'
         , 'awaw' , 'ikowaw','awawok','ikowawok'
         ,'ew','ewok','imew','imewok','imeriw','iko','ikowok','eriw','eriwa','ikoriw','ikoriwa']
vta_03 = ['itai', 'ititai','itan', 'itinan','itawaw', 'itinaw','atanano','ikotanano','atananowok','ikotananowok'
         'atai', 'ikotai','ataiik','ikoktaiik','atan','ikotan','atananak','ikotananak'
         'atawaw','ikotawaw','atawawok','ikotawawok','epan','epanak','ikopan','ikowapanak'
         ,'eripan','eripana','ikoripan','ikor']
vta_09 = ['inatake' , 'inanatake', 'inawatake', 'itinatake','itinanatake'
         ,'itinawatake','ananotake','ananotakenak','ikonanotake','ikonanotakenak'
         ,'atake', 'atakenak' , 'ikonatakenak', 'ananatake','ananatakenak'
         ,'ikonanatakenak','awatake','awatakenak','ikotake','ikowatake','ikotakenak',
         'ikowatakenak','etake','etakena','ikotakena','eritake','eritakena','ikoritake','ikoritakena']
vta_10 = ['inakopan', 'inanakopan','inawakopan','itinakopan','itinanakopan'
         ,'itinawakopan','ananokopan','ananokopanak','ikonanokopan','ikonanokopanak'
         ,'akopan','akopanak','ikokopan','ikokopanak','akopan','ananakopank','ikonanakopan'
         ,'ananakopanak','ikonanakopanak','awakopan','awakopanak','ikokopan'
         ,'ikokopanak','ikowakopan','ekopan','ekopana','ikokopan','ikokopan','ikowakopan',
         'ikowakopana','erikopan','erikopana','ikorikopan','ikorikopana']

vta_11 = ['iin','iak','iekw','itan','itak','itakok','imeko',
         'okw','okok','imoko','imitako','itokw','itokok','imitoko'
         ,'ak','akik','imaki','itc','itcik','imitci','akitc','imitc'
         ,'akitcik','imitcik','imakitci','imtci','atc','iskw',
         'atcik','imatci','iskik','imiski','itakw','itakok','ekw','ekok',
         'atc','atcik','imatc','imatcik','imaritci','eritci','ewaritci',
         'ikoritci','ikowaritci','kotc','ikotcik']

vta_12 = ['iine','itaane','iake','iekwe','itake','itakokwe',
         'okwe','okwawe','itokwe','itokwawe','amake','akawe',
         'ikote','ikowate','imite','imitawe','ate','akite','atawe','iske','iskawe'
         ,'ewekwe','etawe','itakwe','itakawe','amate','awate','ikote','ikowate',
         'arite','ariwate','ariwatena','ikorite','ikoriwate']
vta_13 = ['ipan','iekopan','iakipan','iekopan','itapan','itakipan','itakokipan'
         , 'itakipan','okopan','okwapan','itakopan','itokwapan','akipan','akwapan','ispan',
         'iwaspan','akitipan','akitwapan','atipan','atwapan','ewekopan','ewekwapan','aspan',
         'aspana','awaspan','awaspana','arispan','arispana','imitipan','imitwapan','iskipan',
         'iskwapan','itakopan','itakwapan','ikospan','ikowaspana',
         'ikorispan','ikorwiwarispan']
vta_14 = ['iwonen','iwakwen','iwekwen','itawonen','itakokwen','owokwen', 'owokwenak'
         ,'awoken','awokenak','awakiten','awakitenak','awoten','awotenak','ewekwen','ewekwenak',
         'akwen','awakwen','itokwen','itokwenak','ikwen','iwakwen','itkowen','itokwenak',
         'iskwen','iskwaken','itakwen','itawenak','ikokwen','ikowakwen',
         'ikowakwen','arikwen','arikwena','ikorikwen','ikorikwena']

vta_15 = ['iwopanen','iakopanen','iekopanen','itawopanen','itakopanen',
         'owokopanen','awokopanenak','awokipanen','awokipanenak','awokitipanen',
         'awokitipanenak','awotipanen','awotipanenak','ewekopanen','ewekopanenak',
         'ikokopanenak','ikokopanen','iskwakopanen','iskopanen','iamitokopanenak','iamitokopanen',
         'iwakopanen','ikopanen','itokokopanenak','itokokopanen']

vta_17_a = ['ici','icinan','icikw','imik','imakw','atan','atanak','imatan','i','ik','im','akw','akok']
vta_17_b  = ['icikan','icikanan','icikekw','imakanak','imakan','akan','akanak','imakan','akekw','akekok']


vti_01 = ['en','enawaw','enano','enan','am','amok','amiriw','amiriwa','w','wa','riw','riwawa']
vti_03 = ['etai','etawaw','etano','etai','etan','amopan','amopank','amiripan','amiripana','pan','pana','ripan','ripana']
vti_09 = ['enatake','enawatake','enanotake','amotake','amotakenak','amiritake',
         'amiritakena','take','takena','ritakena','ritakena']
vti_10 = ['enakopan','enawakopan','enanokopan','enanakopan','amokopan','amowakopan','amirikopan','amirikopana']
vti_11 = ['aman','amekw','amokw','amak','ak','akik','amiritci','k','ki','riki']
vti_12a = ['amane','amekwe','amiekwe','amokwe','amoikwe','amane','amake','amane','amote','amowate','amirite','amiritena']
#vti_12b = vit_12a
vti_13 = ['amopan','amekopan','amokopan','amapan','amakipan','akipan','akwapan',
          'amirispan', 'amirispana']
vti_14 = ['amowonen','amwewokwen','amowokwen','amowaken','amokwen','amowakwen','amirikwen','amirikwena']
vti_15 = ['amowopanen','amokwekopanen','amokopanen','amowapanen','amowakipanen','amokopanen','amowkopanen','amirikopanen','amirikopanena']
vti_17a = ['a','amokw','eta']
vti_17b = ['ekan','amokan']



vti2_01 = ['n','nawaw','nano','nan','w','wok','riw','riwa','o','iw','iwa']
vti2_03 = ['tai','tawaw','tanano','tai','tan','pan','panak','ripan','ripana','pan','ipana','opan','opana']
vti2_09 = ['natake','nawatake','nanotake',
          'nanatake','take','takenak','ritake',
         'ritakena','otake','otakena','oritakena','oritakena']
vti2_10 = ['nakopan','nawakopan','nanokopan','nakopan','kopan','wakopan','rikopan','rikopana']
vti2_11 = ['in','iekw','ikw','ian','iak','tc','tcik','ritci','ki','niki','nik']
vti2_12a = ['ine','iekwe','ikwe','iane','iake','te','wate','rite','ritena']
#vti_12b = vit_12a
vti_13 = ['ipan','iekopan','ikopan','iapan','iakipan','span','waspan',
          'rispan', 'rispana']
vti_14 = ['wonen','wewokwen','wokwen','wanen','waken','kwen','wakwen','rikwena','rikwena']
vti_15 = ['wopanen','wekopanen','wokopanen','wapanen','wakipanen','kopanen','wakopanen','rikopanen','rikopanena']
vti_17a = ['kw','tan']
vti_17b = ['kan','kekw','nano']

In [ ]:
 # possessive Pronouns
    #befor w or constant 
    
    #mine , yours , he's
    possisive_pronouns_prefexis = ['ki','ni','o']
    # befor o 
    possisive_pronouns_prefexis = ['k','n','o']
    # before vowel
    possisive_pronouns_prefexis = ['kit','nit','ot']
    
    # notre mine and hes = ni possisive
    ni + word + in/an
    # notre notre  mine and yours 
    ki + word + in/o
    # your (plural) 
    ki + word + iwaw
    #thier 
    o + word + iwaw
    o+ word + iwa 
    o + word + aw
    # his other froms 
    o + word + iriw
    
    # someone propoerty  indefinite format 
    o + word + nan or awik
    
    #surboviatif 
    
    #possisive comes befor the plural
    
                
    stop-words = ['kotahik','awihik','kotak','awik', 'wirawaw','wir','ninan',
                  'kirano','kirawaw','nin','kir' , 'kekwan']
    ask_question_stop_words =['awin','awiritake','kekwan' ,'tan' ]
    presentative_stop_words = ['enko','namaiew']
    alternative_stop_words = ['kotak']
    demonstrative_stop_words =['nahwe','ohki' , 'nohwe', 'niheriw','neheriw', 'neriw']
    
    
    #verbs